In [ ]:
from data_loader import ImgDataset,TgtImgDataset, VideoDataset
from sampler import RandomIdentityBatchSampler,RandomTrackletBatchSampler,RandomIdentitySampler
from data_manager import Mars,DukeMTMC
from utils import GlobalVar
import torch
from torchvision import transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [ ]:
from torch.nn import init
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    # print(classname)
    if classname.find('Conv') != -1:
        # For old pytorch, you may use kaiming_normal.
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_out')
        init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm1d') != -1:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)


def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        init.normal_(m.weight.data, std=0.001)
        init.constant_(m.bias.data, 0.0)

# Defines the new fc layer and classification layer
# |--Linear--|--bn--|--relu--|--Linear--|


class ClassBlock(nn.Module):
    def __init__(self, input_dim, class_num, droprate, relu=False, bnorm=True, num_bottleneck=512, linear=True, return_f=False):
        super(ClassBlock, self).__init__()
        self.return_f = return_f
        add_block = []
        if linear:
            add_block += [nn.Linear(input_dim, num_bottleneck)]
        else:
            num_bottleneck = input_dim
        if bnorm:
            add_block += [nn.BatchNorm1d(num_bottleneck)]
        if relu:
            add_block += [nn.LeakyReLU(0.1)]
        if droprate > 0:
            add_block += [nn.Dropout(p=droprate)]
        add_block = nn.Sequential(*add_block)
        add_block.apply(weights_init_kaiming)

        classifier = []
        classifier += [nn.Linear(num_bottleneck, class_num)]
        classifier = nn.Sequential(*classifier)
        classifier.apply(weights_init_classifier)

        self.add_block = add_block
        self.classifier = classifier

    def forward(self, x):
        x = self.add_block(x)
        if self.return_f:
            f = x
            x = self.classifier(x)
            return x, f
        else:
            x = self.classifier(x)
            return x

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self,num_parts):

        super(FeatureExtractor,self).__init__()
        self.num_parts = num_parts
        resnet50 = models.resnet50(pretrained=True)
        self.base = nn.Sequential(*list(resnet50.children())[:-2])
        self.pcb_pool = nn.AdaptiveAvgPool2d((num_parts,1))
        self.half_pool = nn.AdaptiveAvgPool2d((2,1))
        self.global_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self,x):

        x = self.base(x)

        glob = self.global_pool(x)
        glob = glob.view(glob.size(0),glob.size(1),glob.size(2))

        if self.num_parts > 0:
        
            halfs = self.half_pool(x)
            halfs = halfs.view(halfs.size(0),halfs.size(1),halfs.size(2))
            
            stripes = self.pcb_pool(x)
            stripes = stripes.view(stripes.size(0),stripes.size(1),stripes.size(2))
            
            features = torch.cat([glob,halfs,stripes],dim=2)

        else:
            features = glob
        
        return features

In [ ]:
feature_extractor = FeatureExtractor(6).cuda(3)

In [ ]:
x = torch.randn(4,3,256,128).cuda(3)

In [ ]:
y = feature_extractor(x)
print(y.shape)

In [ ]:
class SrcReidModel(nn.Module):
    def __init__(self,num_classes):
        super(SrcReidModel,self).__init__()
        self.num_parts = feature_extractor.num_parts        
        self.classifier1 = ClassBlock(2048,class_num=num_classes,num_bottleneck=512,droprate=0.5)
        if self.num_parts > 0:
            for i in range(self.num_parts+2): 
                setattr(self,'classifier'+str(i+2),ClassBlock(2048,class_num=num_classes,num_bottleneck=256,droprate=0.5))

    def forward(self,x):
        
        y = []
        for i in range(features.size(2)):
            y.append(getattr(self,'classifier'+str(i+1))(x[:,:,i]))

        return y

In [ ]:
a = torch.randn(4,2048,1)
b = torch.randn(4,2048,6)
c = torch.cat([a,b],dim=2)
print(c.shape)

In [ ]:
src_reid_model = SrcReidModel(615).cuda(3)

In [ ]:
features = feature_extractor(x)
y = src_reid_model(features)
print(len(y))
print(y[0].shape)

In [ ]:
class TgtPartModel(nn.Module):
    def __init__(self,num_parts):
        super(TgtPartModel,self).__init__()

        self.num_parts = num_parts

        if self.num_parts > 0:
            for i in range(self.num_parts): 
                setattr(self,'classifier'+str(i+1),ClassBlock(2048,class_num=self.num_parts,num_bottleneck=256,droprate=0.5))
    def forward(self,x):
        
        y = []
        for i in range(3,x.size(2)):
            y.append(getattr(self,'classifier'+str(i-2))(x[:,:,i]))

        return y

In [ ]:
tgt_part_model = TgtPartModel(feature_extractor.num_parts).cuda(3)

In [ ]:
y = tgt_part_model(features)

In [ ]:
print(y)

In [ ]:
from torch.nn import MSELoss
from torch.nn import CrossEntropyLoss
from torch.nn import TripletMarginLoss

In [ ]:
class ModifiedTripletLoss(nn.Module):
    """Triplet loss with hard positive/negative mining.
    
    Reference:
        Hermans et al. In Defense of the Triplet Loss for Person Re-Identification. arXiv:1703.07737.
    
    Imported from `<https://github.com/Cysu/open-reid/blob/master/reid/loss/triplet.py>`_.
    
    Args:
        margin (float, optional): margin for triplet. Default is 0.3.
    """
    
    def __init__(self, margin=0.3):
        super(ModifiedTripletLoss, self).__init__()
        self.margin = margin
        self.ranking_loss = nn.MarginRankingLoss(margin=margin)
 
    def forward(self,src_input,tgt_input,tracklet_label):
        """
        Args:
            inputs (torch.Tensor): feature matrix with shape (batch_size, feat_dim).
            targets (torch.LongTensor): ground truth labels with shape (num_classes).
        """
        if not isinstance(tracklet_label,torch.Tensor):
            tracklet_label = torch.Tensor(tracklet_label)
        if len(tracklet_label.shape) > 1:
            tracklet_label = torch.flatten(tracklet_label)
        m = src_input.size(0)
        n = tgt_input.size(0)

        tgt_distmat = torch.pow(tgt_input,2).sum(dim=1,keepdim=True).expand(n,n) + \
                      torch.pow(tgt_input,2).sum(dim=1,keepdim=True).expand(n,n).t()
        tgt_distmat.addmm_(1,-2,tgt_input,tgt_input.t())
        tgt_distmat = tgt_distmat.clamp(min=1e-12).sqrt() 
        tgt_src_dismat = torch.pow(tgt_input,2).sum(dim=1,keepdim=True).expand(n,m) + \
                         torch.pow(src_input,2).sum(dim=1,keepdim=True).expand(m,n).t()
        tgt_src_dismat.addmm_(1,-2,tgt_input,src_input.t())
        tgt_src_dismat = tgt_src_dismat.clamp(min=1e-12).sqrt() 

        mask = tracklet_label.expand(n,n).eq(tracklet_label.expand(n,n).t())
        dist_ap,dist_an = [], []
        for i in range(n):
            dist_ap.append(tgt_distmat[i][mask[i]].max().unsqueeze(0))
            dist_an.append(tgt_src_dismat[i].min().unsqueeze(0))
        dist_ap = torch.cat(dist_ap)
        dist_an = torch.cat(dist_an)
        
        # Compute ranking hinge loss
        y = torch.ones_like(dist_an)
        return self.ranking_loss(dist_an, dist_ap, y)


In [ ]:
class TripletLoss(nn.Module):
    """Triplet loss with hard positive/negative mining.
    
    Reference:
        Hermans et al. In Defense of the Triplet Loss for Person Re-Identification. arXiv:1703.07737.
    
    Imported from `<https://github.com/Cysu/open-reid/blob/master/reid/loss/triplet.py>`_.
    
    Args:
        margin (float, optional): margin for triplet. Default is 0.3.
    """
    
    def __init__(self, margin=0.3,global_feat, labels):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.ranking_loss = nn.MarginRankingLoss(margin=margin)
 
    def forward(self, inputs, targets):
        """
        Args:
            inputs (torch.Tensor): feature matrix with shape (batch_size, feat_dim).
            targets (torch.LongTensor): ground truth labels with shape (num_classes).
        """
        n = inputs.size(0)
        
        # Compute pairwise distance, replace by the official when merged
        dist = torch.pow(inputs, 2).sum(dim=1, keepdim=True).expand(n, n)
        dist = dist + dist.t()
        dist.addmm_(1, -2, inputs, inputs.t())
        dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
        
        # For each anchor, find the hardest positive and negative
        mask = targets.expand(n, n).eq(targets.expand(n, n).t())
        dist_ap, dist_an = [], []
        for i in range(n):
            dist_ap.append(dist[i][mask[i]].max().unsqueeze(0))
            dist_an.append(dist[i][mask[i] == 0].min().unsqueeze(0))
        dist_ap = torch.cat(dist_ap)
        dist_an = torch.cat(dist_an)
        
        # Compute ranking hinge loss
        y = torch.ones_like(dist_an)
        return self.ranking_loss(dist_an, dist_ap, y)


In [ ]:

modified_triplet_loss = ModifiedTripletLoss()
a = torch.randn(4,2048,9)
b = torch.randn(4,2048,9)

loss = modified_triplet_loss(a[:,:,0],b[:,:,0],[2,2,3,4])

In [ ]:
a = torch.randn(3,4)
b = torch.flatten(a)
print(b.shape)

In [ ]:
print(loss)

In [ ]:
glob_var = GlobalVar()
mars = Mars()
dukemtmc = DukeMTMC()

In [59]:

train_transform = T.Compose(
    [
        T.Resize((256,128)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]
)
train_transform = T.Compose(
    [
        T.Resize((256,128)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]
)
tgt_train_dataset = VideoDataset(mars.train,train_transform,4)
tgt_train_loader = DataLoader(tgt_train_dataset, \
                   batch_sampler = RandomTrackletBatchSampler(dataset=mars.train,glob_var=glob_var),
                   num_workers = 0,
                   pin_memory = False,
                   collate_fn=tgt_collate_fn,
                   num_workers=4)

src_train_dataset = ImgDataset(dukemtmc.train,train_transform)
tgt_img_dataset = TgtImgDataset(mars.gallery,train_transform)
src_reid_loader = DataLoader(dataset = src_train_dataset, batch_size = 64,shuffle=False,num_workers=4,drop_last=False)

SyntaxError: keyword argument repeated (<ipython-input-59-3b8ea5c3ff1f>, line 23)

In [ ]:
glob_var.tgt_batch_size = 16
tgt_train_iter = iter(tgt_train_loader)

In [ ]:
src_reid_iter = iter(src_reid_loader)
print(next(src_reid_iter))

In [ ]:
mars.train.loc[:,'pid'] = 1

In [ ]:
print(next(tgt_train_iter))

In [ ]:
def tgt_collate_fn(batch):
    imgs = []
    pids = []
    camids = []
    tracklets = []
    paths = []
    for img,pid,camid,path,tracklet in batch:
        imgs += [img.unsqueeze(dim=0)]
        pids += pid
        camids += camid
        tracklets += tracklet
        paths += path
    imgs = torch.cat(imgs,dim=0)
    b,t,c,h,w = imgs.shape
    imgs = imgs.view(b*t,c,h,w)
    return imgs,torch.IntTensor(pids),torch.IntTensor(camids),paths,torch.IntTensor(tracklets)

In [55]:
def extract_feature(dataloader, feature_extractor):
    features = []
    pids = []
    camids = []
    tracklets = []
    paths = []
    with torch.no_grad():
        for batch_idx,batch in enumerate(dataloader):
            imgs = batch[0].cuda(3)
            pid = batch[1]
            camid = batch[2]
            path = list(batch[3])
            if len(batch) > 4:
                tracklet = batch[4]
                tracklets.append(tracklet)

            feature = feature_extractor(imgs)

            features += [feature[:,:,0].cpu()] 
            pids += [pid]
            camids += [camid]
            paths += path
            print(batch_idx)
        features = torch.cat(features,dim=0)
        pids = torch.cat(pids,dim=0).numpy()
        if len(tracklets) > 0:
            tracklets = torch.cat(tracklets,dim=0).numpy()
    return features,pids,camids,paths,tracklets
    

In [ ]:
a = torch.randn(4,3)
b = torch.randn(3,3)
c = torch.cat([a,b],dim=0)
print(c.numpy())

In [ ]:
a = (1,2,3)
b = list(a)

In [ ]:
print(b)

In [ ]:
def compute_distmat(a,b):
    m = a.size(0)
    n = b.size(0)
    distmat = torch.pow(a,2).sum(dim=1,keepdim=True).expand(m,n) + \
                         torch.pow(b,2).sum(dim=1,keepdim=True).expand(n,m).t()
    distmat.addmm_(1,-2,a,b.t())
    distmat = distmat.clamp(min=1e-12).sqrt() 
    return distmat.numpy()

In [ ]:
print(result[0].shape)

In [ ]:
distmat = compute_distmat(result[0],result[0])


In [ ]:
print(distmat.shape)

In [ ]:
glob_var.feature_extractor = feature_extractor
glob_var.save_path = './Logs'
glob_var.src_data = DukeMTMC()
glob_var.tgt_data = Mars()


In [62]:
class Stage1Var():
    def init():
        reid_batchsize = 64
        part_batchsize = 64
        num_workers = 4
        use_gpu = True
        transfrom = T.Compose(
                    [
                        T.Resize((256,128)),
                        T.RandomHorizontalFlip(),
                        T.ToTensor(),
                        T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
                    ]
                )
        



glob_var.stage1 = Stage1Var()
glob_var.stage1.sampler = RandomIdentitySampler(glob_var.src_data.train,glob_var.seq_len)

In [ ]:
def TrainStage1(object):
    def __init__(self,glob_var):
        self.glob_var = glob_var
        self.feature_extractor = glob_var.feature_extractor
        self.save_freq = 0
        self.

    def get_dataloader(self,glob_var):
        src_loader = DataLoader(ImgDataset(glob_var.src_data.train,glob_var.stage1.transform),
                               batch_size = glob_var.stage1.reid_datasize,
                               num_workers=glob_var.stage1.num_workers,
                               sampler=glob_var.stage1.smapler,
                               drop_last=True,
                               pin_memory=False
                               )
        tgt_loader = DataLoader(TgtImgDataset(glob_var.tgt_data.train,glob_var.satge1.transform),
                                    )
        
    def train(self):
        pass
            
def TrainStage2(object):
    def __init__(self,glob_var):
        self.glob_var = glob_var
    def get_dataloader(self,)